In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib
import torch
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from PIL import Image
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

from PIL import Image
#from albumentations.pytorch import ToTensor 
from torch.utils.data import Dataset, random_split
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import sys
import sklearn

import cv2
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from torchvision import models, transforms
from torch.utils.data import WeightedRandomSampler

import matplotlib.pyplot as plt



IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)


'''
IMAGE TRANSFORMS
'''
class ImageTransform:
    def __init__(self, size, mean, std):
        self.data_transform = transforms.Compose([
                transforms.Resize((size, size), interpolation=Image.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

    def __call__(self, img):
        return self.data_transform(img)


In [3]:
transformer = ImageTransform(IMG_SIZE, mean, std)

In [4]:
device = 'cuda'
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
model = model.to(device)

# Load the model state
deepfake_types = ['Deepfakes', 'Face2Face', 'FaceShifter', 'FaceSwap', 'NeuralTextures']

Loaded pretrained weights for efficientnet-b0


In [5]:
def test_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    total_loss = 0.0
    correct = 0
    total = 0
    criterion = torch.nn.BCEWithLogitsLoss()  # Binary classification loss

    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move data to the correct device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Model forward pass
            outputs = model(inputs)
            
            # Ensure outputs and labels have the same shape
            outputs = outputs.view(-1)  # Flatten outputs to match labels
            labels = labels.view(-1).float()  # Flatten labels and convert to float

            # Compute loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Calculate predictions and accuracy
            predictions = (torch.sigmoid(outputs) > 0.5).long()
            correct += (predictions == labels.long()).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
deepfakes_test_statistics = {}
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
model = model.to(device)
checkpoint = torch.load(f"ff_effnetb0_global.pth")
model.load_state_dict(checkpoint["model_state"])
for deepfake_type_dataset in deepfake_types:
    DEEPFAKE_TYPE = deepfake_type_dataset
    TEST_FOLDER = fr'D:\FF\crops\{DEEPFAKE_TYPE}\test' 
    test_ds = ImageFolder(root=TEST_FOLDER, transform=transformer)
    test_loader = DataLoader(test_ds, shuffle=False, batch_size=64)

    avg_loss, accuracy = test_model(model, test_loader)
    print(avg_loss, accuracy)
    deepfakes_test_statistics[deepfake_type_dataset] = accuracy
        

Loaded pretrained weights for efficientnet-b0


C:\Users\Sheryl\AppData\Local\Temp\ipykernel_25216\3520496489.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"ff_effnetb0_global.pth")


0.42562573301096984 0.8970290344361918
0.42562573301096984 0.8970290344361918
0.42562573301096984 0.8970290344361918
0.42562573301096984 0.8970290344361918
0.42562573301096984 0.8970290344361918
0.5160294544983767 0.8829081919316777
0.5160294544983767 0.8829081919316777
0.5160294544983767 0.8829081919316777
0.5160294544983767 0.8829081919316777
0.5160294544983767 0.8829081919316777
0.5433410690654696 0.8626392170097874
0.5433410690654696 0.8626392170097874
0.5433410690654696 0.8626392170097874
0.5433410690654696 0.8626392170097874
0.5433410690654696 0.8626392170097874
0.3684050060985328 0.9248061144205912
0.3684050060985328 0.9248061144205912
0.3684050060985328 0.9248061144205912
0.3684050060985328 0.9248061144205912
0.3684050060985328 0.9248061144205912
1.3203227198757512 0.7504498425551057
1.3203227198757512 0.7504498425551057
1.3203227198757512 0.7504498425551057
1.3203227198757512 0.7504498425551057
1.3203227198757512 0.7504498425551057


In [8]:
deepfakes_test_statistics

{'Deepfakes': {'Deepfakes': 0.8970290344361918,
  'Face2Face': 0.8970290344361918,
  'FaceShifter': 0.8970290344361918,
  'FaceSwap': 0.8970290344361918,
  'NeuralTextures': 0.8970290344361918},
 'Face2Face': {'Deepfakes': 0.8829081919316777,
  'Face2Face': 0.8829081919316777,
  'FaceShifter': 0.8829081919316777,
  'FaceSwap': 0.8829081919316777,
  'NeuralTextures': 0.8829081919316777},
 'FaceShifter': {'Deepfakes': 0.8626392170097874,
  'Face2Face': 0.8626392170097874,
  'FaceShifter': 0.8626392170097874,
  'FaceSwap': 0.8626392170097874,
  'NeuralTextures': 0.8626392170097874},
 'FaceSwap': {'Deepfakes': 0.9248061144205912,
  'Face2Face': 0.9248061144205912,
  'FaceShifter': 0.9248061144205912,
  'FaceSwap': 0.9248061144205912,
  'NeuralTextures': 0.9248061144205912},
 'NeuralTextures': {'Deepfakes': 0.7504498425551057,
  'Face2Face': 0.7504498425551057,
  'FaceShifter': 0.7504498425551057,
  'FaceSwap': 0.7504498425551057,
  'NeuralTextures': 0.7504498425551057}}

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np



# Convert data to a numpy array for heatmap
labels = list(deepfakes_test_statistics.keys())
values = np.array([[deepfakes_test_statistics[j][i] for j in deepfake_types] for i in deepfake_types])

# Create the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(values, annot=True, cmap="YlGnBu", xticklabels=deepfake_types, yticklabels=deepfake_types, fmt=".2f")
plt.title("Model Performance against Datasets")
plt.xlabel("Dataset Type")
plt.ylabel("Model Type")
plt.show()

# knowledge distillation type learning
# soumik benchmark on company deepfake detector - because detectors are created on model structure
# note down amount of training data in the test set and dataset
# different loss functions - focal loss, 
# implement federated learning averaging gradients function logic
# foundational model for all datasets
# check other models